In [1]:
from fastai2.basics import *
from fastai2.vision.all import *
from faststyle import *

In [2]:
source = untar_data(URLs.COCO_SAMPLE)

In [3]:
style_dir = Path('styles')
style_fns = L(style_dir/'abstract.jpg', style_dir/'starry_night.jpg')

In [4]:
class TensorImage1(TensorImage): pass

In [5]:
class PILImage1(PILImage): pass
PILImage1._tensor_cls = TensorImage1

In [6]:
class NormalizeInp(Transform):
  def __init__(self, norm): self.norm = norm
  def encodes(self, x:TensorImage1, **kwargs): return self.norm.encodes(x, **kwargs)
  def decodes(self, x:TensorImage1, **kwargs): return self.norm.decodes(x, **kwargs)
  @classmethod
  def from_stats(cls, *args, **kwargs): 
    return cls(Normalize.from_stats(*args, **kwargs))

In [7]:
dblock = DataBlock((ImageBlock(PILImage1), ImageBlock), get_items=get_image_files, splitter=RandomSplitter(.1),
                   item_tfms=[Resize(128)],
                   batch_tfms=NormalizeInp.from_stats(*coco_stats))

In [8]:
dls = dblock.dataloaders(source, bs=16)

In [9]:
get_feats = LayerFeats.from_feat_m(FeatModels.vgg19)

In [10]:
source_tims = L(TensorImage.create(fn) for fn in style_fns)

In [11]:
# source_tims = Normalize.from_stats(*imagenet_stats)(source_tims)

In [12]:
stl_fts,cnt_fts = zip(*L(get_feats(o[None]) for o in source_tims))
stl_fts,cnt_fts = L(zip(*stl_fts)),L(zip(*cnt_fts))

In [13]:
cbs = [SourceFeatsCallback(stl_fts, cnt_fts)]

In [14]:
loss_func = FastStyleLoss2(get_feats)

In [15]:
learn = style_learner(dls, style_fns, loss_func=loss_func, cbs=cbs)

In [16]:
# learn.lr_find()

In [17]:
learn.fit_one_cycle(1, 1e-3)

KeyboardInterrupt: 

In [ ]:
inp,pred,dec = learn.get_preds(with_input=True, with_decoded=True)

In [ ]:
learn.show_results()